<a href="https://colab.research.google.com/github/glaucohb/DSWP/blob/master/Desafio_Redes_Neurais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Passo 1 - Importar os dados
Pasta dos dados = C:\Users\02549299744\Desktop\Python_Sufis\Aulas Nélio_Desafio RFB

Arquivos treinamento = https://raw.githubusercontent.com/glaucohb/DSWP/master/Dataframes/desafio_train.csv


Arquivos teste = https://raw.githubusercontent.com/glaucohb/DSWP/master/Dataframes/desafio_test.csv

In [34]:
import pandas as pd
X_treinamento_original = pd.read_csv('https://raw.githubusercontent.com/glaucohb/DSWP/master/Dataframes/desafio_train.csv', index_col ='id')
X_teste = pd.read_csv('https://raw.githubusercontent.com/glaucohb/DSWP/master/Dataframes/desafio_test.csv')
X_treinamento_original.shape

(11033, 62)

In [35]:
X_treinamento_original.head()

,cnae2,rf2,md1,md2,md3,md4,md5,md6,md7,md8,md9,md10,md11,md12,mc1,mc2,mc3,mc4,ind01,ind02,ind03,ind04,ind05,ind06,ind07,ind08,ind09,ind10,ind11,ind12,ind13,ind14,ind15,ind16,ind17,ind18,ind19,ind20,ind21,ind22,ind23,ind24,ind25,ind26,ind27,ind28,ind29,ind30,ind31,ind32,ind33,ind34,ind35,ind36,ind37,ind38,ind39,ind40,ind41,ind42,ind43,target
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,86,i,0.090339,0.011256,0.111095,0.003233,0.003233,0.0,0.016366,0.021082,0.004541,0.004541,0.0,0.130930,0.009247,0.001729,0.000138,0.004412,0.0,0.0281,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,1.0,0.0000,0.00,0.00,0.0000,0.0000,0.0976,0.0333,0.1000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,True
1,18,z,0.005996,0.019476,0.124770,0.000000,0.038168,0.0,0.029214,0.046445,0.000000,0.049919,0.0,0.149741,0.003186,0.007671,0.006943,0.003589,0.0,0.0000,1.0,0.4167,0.4194,0.7068,0.0076,0.0076,0.0,0.0,0.0,0.0856,0.0286,0.0,0.0,0.0,0.0,0.0,0.0,0.7625,1.00,1.00,0.1429,0.2857,0.0000,0.4444,0.2222,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
2,74,z,0.000006,0.002902,0.110160,0.002274,0.002274,0.0,0.002902,0.020058,0.003131,0.003131,0.0,0.130405,0.000015,0.000000,0.000050,0.000209,0.0,0.0000,1.0,1.0000,1.0000,1.0000,0.0000,0.0000,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,1.0,1.0000,1.00,1.00,1.0000,1.0000,0.0000,1.0000,1.0000,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,False
3,49,q,0.000009,0.014526,0.120351,0.000000,0.000000,0.0,0.014526,0.032017,0.000000,0.000000,0.0,0.142568,0.000438,0.000000,0.000042,0.000115,0.0,0.0000,0.0,0.0000,0.0000,0.0000,0.2309,0.2309,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,1.0000,0.25,0.25,0.9167,0.9167,0.7857,0.1667,0.1667,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,False
4,47,i,0.000191,0.004042,0.111078,0.001121,0.001121,0.0,0.006063,0.023705,0.011886,0.011886,0.0,0.131285,0.001925,0.004829,0.006355,0.000719,0.0,0.0000,0.0,0.0000,0.0000,0.0000,1.0000,1.0000,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.00,0.00,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.5,False


In [36]:
X_teste.shape

(1000, 62)

In [37]:
X_teste.head()

,id,cnae2,rf2,md1,md2,md3,md4,md5,md6,md7,md8,md9,md10,md11,md12,mc1,mc2,mc3,mc4,ind01,ind02,ind03,ind04,ind05,ind06,ind07,ind08,ind09,ind10,ind11,ind12,ind13,ind14,ind15,ind16,ind17,ind18,ind19,ind20,ind21,ind22,ind23,ind24,ind25,ind26,ind27,ind28,ind29,ind30,ind31,ind32,ind33,ind34,ind35,ind36,ind37,ind38,ind39,ind40,ind41,ind42,ind43
0,3411,71,s,0.017485,0.004743,0.111771,0.003121,0.003121,0.0,0.005432,0.023085,0.009890,0.011346,0.0,0.131320,0.000176,0.000000,1.442161e-09,0.000917,0.0,0.0326,0.0,1.0,1.0,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0000,0.75,0.75,1.0,1.0,0.0625,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2177,86,y,0.012269,0.005919,0.111803,0.001136,0.000000,0.0,0.006160,0.022035,0.001686,0.000000,0.0,0.132734,0.005597,0.004738,8.343705e-06,0.001462,0.0,0.0135,0.0,0.0,0.0,1.0,1.0000,1.0000,0.0,0.0,0.0,0.0293,0.5883,0.1016,0.0,0.0,0.0,0.0,0.0000,0.0000,1.00,0.00,0.0,0.0,0.0000,1.0000,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,8400,41,z,0.002325,0.001882,0.109870,0.000000,0.000000,0.0,0.002823,0.020522,0.000000,0.000000,0.0,0.131390,0.000206,0.000000,7.473043e-05,0.000659,0.0,0.0438,0.0,1.0,1.0,1.0,0.0000,0.0000,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,1.0000,1.0000,0.00,0.00,1.0,1.0,0.0000,1.0000,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,464,58,z,0.022149,0.042115,0.143274,0.052107,0.005284,0.0,0.042115,0.056179,0.044911,0.004436,0.0,0.145176,0.003389,0.000693,1.104422e-03,0.003108,0.0,0.0000,0.0,0.0,0.0,0.0,0.4810,0.8654,0.0,0.0,0.0,0.3209,0.1926,0.0000,0.0,0.0,0.0,0.0,0.8694,0.9212,0.75,0.75,0.0,0.0,0.3091,0.1389,0.5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,6672,86,p,0.011190,0.004536,0.110678,0.002873,0.003131,0.0,0.004536,0.020658,0.003853,0.004202,0.0,0.130692,0.000763,0.000000,0.000000e+00,0.000387,0.0,0.0153,0.0,0.0,0.0,0.0,0.7799,0.7799,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0000,1.0000,0.00,0.00,0.0,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


## Criar o dataframe X_treinamento

In [38]:
X_treinamento = X_treinamento_original.copy()
X_treinamento.head(3)

,cnae2,rf2,md1,md2,md3,md4,md5,md6,md7,md8,md9,md10,md11,md12,mc1,mc2,mc3,mc4,ind01,ind02,ind03,ind04,ind05,ind06,ind07,ind08,ind09,ind10,ind11,ind12,ind13,ind14,ind15,ind16,ind17,ind18,ind19,ind20,ind21,ind22,ind23,ind24,ind25,ind26,ind27,ind28,ind29,ind30,ind31,ind32,ind33,ind34,ind35,ind36,ind37,ind38,ind39,ind40,ind41,ind42,ind43,target
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,86,i,0.090339,0.011256,0.111095,0.003233,0.003233,0.0,0.016366,0.021082,0.004541,0.004541,0.0,0.130930,0.009247,0.001729,0.000138,0.004412,0.0,0.0281,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,1.0,0.0000,0.0,0.0,0.0000,0.0000,0.0976,0.0333,0.1000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,True
1,18,z,0.005996,0.019476,0.124770,0.000000,0.038168,0.0,0.029214,0.046445,0.000000,0.049919,0.0,0.149741,0.003186,0.007671,0.006943,0.003589,0.0,0.0000,1.0,0.4167,0.4194,0.7068,0.0076,0.0076,0.0,0.0,0.0,0.0856,0.0286,0.0,0.0,0.0,0.0,0.0,0.0,0.7625,1.0,1.0,0.1429,0.2857,0.0000,0.4444,0.2222,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
2,74,z,0.000006,0.002902,0.110160,0.002274,0.002274,0.0,0.002902,0.020058,0.003131,0.003131,0.0,0.130405,0.000015,0.000000,0.000050,0.000209,0.0,0.0000,1.0,1.0000,1.0000,1.0000,0.0000,0.0000,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,1.0,1.0000,1.0,1.0,1.0000,1.0000,0.0000,1.0000,1.0000,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,False


In [44]:
X_treinamento.isna().sum()

cnae2       0
rf2         0
md1         0
md2         0
md3         0
         ... 
ind40      34
ind41      34
ind42     599
ind43     599
target      0
Length: 62, dtype: int64

In [10]:
a = X_treinamento.dtypes
dfa = pd.DataFrame(a)
dfa

,0
cnae2,int64
rf2,object
md1,float64
md2,float64
md3,float64
...,...
ind39,float64
ind40,float64
ind41,float64
ind42,float64


In [47]:
dfa[dfa[0] == 'int64']

,0
cnae2,int64


In [9]:
(dfa[0] == 'float64').sum()

59

In [13]:
dfa[dfa[0] == 'object']

,0
rf2,object


In [56]:
# Excluir a coluna object para ver se funciona
X_treinamento.drop(['rf2'],  axis=1, inplace = True)


In [45]:
# Excluir os NA
X_treinamento.dropna(inplace= True)

In [46]:
X_treinamento.shape

(10400, 62)

## Criar o dataframe Y_treinamento

In [47]:
Y_treinamento = X_treinamento['target']
X_treinamento.drop(['target'],  axis=1, inplace = True)
Y_treinamento.head()

id
0     True
1    False
2    False
3    False
4    False
Name: target, dtype: bool

In [48]:
Y_treinamento.shape

(10400,)

# Defindo os Neurônios
## Número de neurônios na camada de entrada (Input Layer):  NI 
NI = Número de colunas (ou variáveis) no dataframe.

## Número de neurônios na camada de saída (Output Layer):  NO 
Se a Rede Neural é uma regressão, então o número de neurônios na Output Layer é 1, pois o output de uma regressão é um valor;
Se a Rede Neural é uma classificação e usamos uma função de ativação probabilística (como softmax, por exemplo), então o número de neurônios na Output Layer é igual ao número de classes que queremos prever. Por exemplo, no problema de classificar espécies no dataframe IRIS temos 3 espécies (versicolor, virginica e setosa). Ao utilizarmos a função de ativação softmax, então teremos 3 neurônios na Output Layer.

## Número de neurônios na camada escondida (Hidden Layer):  NH 
No entanto, o artigo How to choose the number of hidden layers and nodes in a feedforward neural network? sugere o uso da seguinte expressão:

NH=N/α(NI+NO) 

onde  N  é o número de instâncias (linhas) do dataframe e  α  é um número entre 2 e 10, sendo que alguns experimentos com  α=2  produzem bons modelos sem overfitting. Para saber mais sobre esta expressão e sobre  α , sugiro a leitura do artigo mencionado.

## LOSS FUNCTION  L 
Como vimos anteriormente, nosso objetivo é minimizar a Loss function através do Gradient Descent. Em outras palavras, esse processo de otimização busca, à cada iteração (epoch), atualizar os pesos  W  para reduzir a Loss Function. As Loss Function mais comuns são:

* Classificação: cross-entropy (quando queremos probabilidades de cada observação pertencer à uma determinada classe).
    * Classificação binária: tf.keras.losses.BinaryCrossentropy();
 
## MÉTRICAS PARA MEDIR A PERFORMANCE DAS REDES NEURAIS
As métricas medem a qualidade/performance das Redes Neurais e as principais são:

* Classificação: Quanto maior a accuracy, melhor a performance da Rede Neural.

    *Accuracy   

## DROPOUT
Dropout significa ignorar aleatoriamente e temporariamente um percentual  p  de neurônios durante a fase de treinamento. Ao "ignorar", quero dizer que tais neurônios não serão considerados durante os processos forward e backpropagation.

Dropout força que a Rede Neural aprenda a partir dos dados, mas usando diferentes e aleatórios neurônios;

* Recomenda-se  p=0.20 .

## Vamos obedecer os steps a seguir para construir nossa Rede Neural:

1. Carregar as bibliotecas do Python e Tensorflow;
2. Carregar os dados para treinar a Rede Neural;
3. Definir a arquitetura da Rede Neural com Tensorflow/Keras;
4. Compilar a Rede Neural;
5. Ajustar a Rede Neural;
6. Avaliar a performance da Rede Neural;
7. _Fine tuning_ da Rede Neural;
8. Fazer Predições com a Rede Neural;
9. Conclusões.

## 1. Carregar as bibliotecas do Python e Tensorflow

In [49]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [50]:
# Asseurar que esta na versão 2.x
tf.__version__

'2.3.0'

In [51]:
# Definir o número de casas decimais = 3
np.set_printoptions(precision = 3)

## 3. Definir a arquitetura

In [61]:
# Número de Neurônios na Input Layer:
N_I = X_treinamento.shape[1] # Número de variáveis/colunas da matriz de preditoras

# Número de neurônios na Output Layer:
N_O = 1

# Número de neurônios na Hidden Layer:
# useia fórmula
N_H = int(X_treinamento.shape[0]/(2*(N_I+N_O)))

# Função de Ativação da Hidden Layer:
FA_H = tf.keras.activations.sigmoid

# Função de Ativação da Output Layer
FA_O = tf.keras.activations.sigmoid

##### compilação

# Algoritmo de otimização
algoritmo_otimizacao = tf.keras.optimizers.Adam() 

# A métrica para cálculo do erro
loss_function = tf.keras.losses.MeanSquaredError() 

metrica_performance = [tf.keras.metrics.binary_accuracy]

In [62]:
RN = Sequential() # nome da Rede Neural
RN.add(Dense(units = N_H, 
             input_dim = N_I, 
             activation = FA_H, 
             kernel_constraint = tf.keras.constraints.UnitNorm()))
RN.add(Dense(units = N_O, activation = FA_O))

# Resumo da arquitetura da Rede Neural:
print(RN.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 85)                5185      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 86        
Total params: 5,271
Trainable params: 5,271
Non-trainable params: 0
_________________________________________________________________
None


4. Compilar a Rede Neural;


In [63]:
RN.compile(optimizer = algoritmo_otimizacao, 
           loss = loss_function, 
           metrics = metrica_performance)

5. Ajustar a Rede Neural;

In [64]:
#  Comando modelo.fit(X_treinamento, y_treinamento, epochs):
hist = RN.fit(X_treinamento, Y_treinamento, epochs = 10)

Epoch 1/10
325/325 [==============================] - 0s 1ms/step - loss: 0.1773 - binary_accuracy: 0.7587
Epoch 2/10
325/325 [==============================] - 0s 1ms/step - loss: 0.1728 - binary_accuracy: 0.7592
Epoch 3/10
325/325 [==============================] - 0s 1ms/step - loss: 0.1713 - binary_accuracy: 0.7586
Epoch 4/10
325/325 [==============================] - 0s 1ms/step - loss: 0.1708 - binary_accuracy: 0.7588
Epoch 5/10
325/325 [==============================] - 0s 1ms/step - loss: 0.1706 - binary_accuracy: 0.7592
Epoch 6/10
325/325 [==============================] - 0s 1ms/step - loss: 0.1699 - binary_accuracy: 0.7602
Epoch 7/10
325/325 [==============================] - 0s 1ms/step - loss: 0.1693 - binary_accuracy: 0.7617
Epoch 8/10
325/325 [==============================] - 0s 1ms/step - loss: 0.1701 - binary_accuracy: 0.7622
Epoch 9/10
325/325 [==============================] - 0s 1ms/step - loss: 0.1688 - binary_accuracy: 0.7629
Epoch 10/10
325/325 [================



6. Avaliar a performance da Rede Neural;
 Comando modelo.evaluate(X_teste, y_teste)

In [65]:
RN.evaluate(X_treinamento, Y_treinamento)

325/325 [==============================] - 0s 838us/step - loss: 0.1677 - binary_accuracy: 0.7640


[0.16773369908332825, 0.7640384435653687]

7. Fine tuning da Rede Neural;

#### Estratégias para melhorar a acurácia da Rede Neural

Nossas alternativas são:

* a. Aumentar o número de neurônios na _Hidden Layer_;
* b. Aumentar o número de _Hidden Layers_;
* c. Aumentar o número de _Hidden Layers_ e o número de neurônios;
* d. Alterar a função de ativação;
* e. Aumentar o número de _epochs_;
* f. Alterar o algoritmo de otimização (_optimizer_);



#### 7.3. Definir a arquitetura da Rede Neural com Tensorflow/Keras

In [71]:
# Número de Neurônios na Input Layer:
N_I = X_treinamento.shape[1] # NÃO FOI ALTERADA!

# Número de neurônios na Output Layer:
N_O = 1 # NÃO FOI ALTERADA!

# VARIÁVEIS ALTERADAS:
# Número de neurônios na Hidden Layer:
N_H = 100

# Função de Ativação da Hidden Layer:
FA_H = tf.keras.activations.relu # ALTERADA!

# Função de Ativação da Output Layer
FA_O = tf.keras.activations.sigmoid # NÃO FOI ALTERADA!

In [72]:
RN = Sequential()
RN.add(Dense(units = N_H, input_dim = N_I, activation = FA_H, kernel_constraint = tf.keras.constraints.UnitNorm()))
RN.add(Dense(units = N_O, activation = FA_O))

print(RN.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 100)               6100      
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 101       
Total params: 6,201
Trainable params: 6,201
Non-trainable params: 0
_________________________________________________________________
None


 ## Compilar a Rede Neural
 

In [73]:
#algoritmo_otimizacao = tf.keras.optimizers.Adam()
algoritmo_otimizacao = tf.keras.optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name= 'Adam')

loss_function = tf.keras.losses.MeanSquaredError()
metrica_performance = [tf.keras.metrics.binary_accuracy]

RN.compile(optimizer = algoritmo_otimizacao, loss = loss_function, metrics = metrica_performance)

#### 7.5. Ajustar a Rede Neural

In [74]:
RN.fit(X_treinamento, Y_treinamento, epochs = 10)

Epoch 1/10
325/325 [==============================] - 0s 1ms/step - loss: 0.2413 - binary_accuracy: 0.7575
Epoch 2/10
325/325 [==============================] - 0s 1ms/step - loss: 0.2411 - binary_accuracy: 0.7564
Epoch 3/10
325/325 [==============================] - 0s 1ms/step - loss: 0.2412 - binary_accuracy: 0.7588
Epoch 4/10
325/325 [==============================] - 0s 1ms/step - loss: 0.2412 - binary_accuracy: 0.7588
Epoch 5/10
325/325 [==============================] - 0s 1ms/step - loss: 0.2412 - binary_accuracy: 0.7587
Epoch 6/10
325/325 [==============================] - 0s 1ms/step - loss: 0.2409 - binary_accuracy: 0.7578
Epoch 7/10
325/325 [==============================] - 0s 1ms/step - loss: 0.2389 - binary_accuracy: 0.7568
Epoch 8/10
325/325 [==============================] - 0s 1ms/step - loss: 0.1808 - binary_accuracy: 0.7575
Epoch 9/10
325/325 [==============================] - 0s 1ms/step - loss: 0.1774 - binary_accuracy: 0.7591
Epoch 10/10
325/325 [================


8. Fazer Predições com a Rede Neural;

Comando RN.predict_classes(X_treinamento):


In [75]:
y_pred = RN.predict_classes(X_treinamento)
y_pred

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]], dtype=int32)

In [76]:
y_pred.shape

(10400, 1)